In [1]:
## GameDataAnalysis

import pandas as pd
import numpy as np 

players = pd.read_csv("D:\\2. PROJECTS\\15. OTHER PRACTICE PROJECTS\\SQL Game Data\\players.csv")
players.head()

,user_id,install_date,country,acquisition_source
0,1,11-01-2025,India,Facebook
1,2,12-01-2025,USA,Facebook
2,3,04-01-2025,India,Organic
3,4,02-01-2025,UK,Organic
4,5,15-01-2025,UK,Facebook


In [2]:
sessions = pd.read_csv("D:\\2. PROJECTS\\15. OTHER PRACTICE PROJECTS\\SQL Game Data\\sessions.csv")
sessions.head()

,user_id,session_date,session_duration
0,1,15-01-2025,29
1,1,13-01-2025,31
2,2,16-01-2025,34
3,3,12-01-2025,8
4,3,14-01-2025,21


In [3]:
purchases = pd.read_csv("D:\\2. PROJECTS\\15. OTHER PRACTICE PROJECTS\\SQL Game Data\\purchases.csv")
purchases.head()

,user_id,purchase_date,amount
0,1,17-01-2025,199
1,1,17-01-2025,199
2,4,11-01-2025,149
3,5,23-01-2025,149
4,5,27-01-2025,299


In [4]:
players[players["country"] == "India"]

,user_id,install_date,country,acquisition_source
0,1,11-01-2025,India,Facebook
2,3,04-01-2025,India,Organic
6,7,01-01-2025,India,Facebook
12,13,13-01-2025,India,Facebook
14,15,05-01-2025,India,Facebook
18,19,05-01-2025,India,Organic
26,27,14-01-2025,India,Facebook
34,35,04-01-2025,India,Google
38,39,04-01-2025,India,Google
41,42,06-01-2025,India,Organic


In [5]:
## ARPU

arpu = purchases.groupby("user_id")["amount"].sum().mean()
arpu

475.9047619047619

In [6]:
revenue_per_user = purchases.groupby("user_id", as_index=False)["amount"].sum()
revenue_per_user.head()

,user_id,amount
0,1,398
1,4,149
2,5,846
3,6,348
4,11,596


In [7]:
## Your current ARPU only averages among users who purchased.
## True ARPU should include all users (including 0 spenders).

arpu = (
    purchases.groupby("user_id")["amount"].sum()
    .reindex(players["user_id"], fill_value=0)
    .mean()
)

arpu


166.56666666666666

In [8]:
## Cohort Analysis

players["install_date"] = pd.to_datetime(players["install_date"])

players["install_month"] = players["install_date"].dt.to_period("M")
cohort = players.groupby(["install_month"])["user_id"].nunique()
cohort

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\2963216353.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  players["install_date"] = pd.to_datetime(players["install_date"])


install_month
2025-01    16
2025-02     3
2025-03     3
2025-04     6
2025-05     2
2025-06     4
2025-07     4
2025-08     4
2025-09     5
2025-10     3
2025-11     4
2025-12     6
Freq: M, Name: user_id, dtype: int64

In [9]:
## Daily KPI report

def daily_kpi_report(sessions, purchases):
    sessions["session_date"] = pd.to_datetime(sessions["session_date"]).dt.date
    purchases["purchase_date"] = pd.to_datetime(purchases["purchase_date"]).dt.date
    
    daily_active_users = sessions.groupby("session_date")["user_id"].nunique()
    daily_revenue = purchases.groupby("purchase_date")["amount"].sum()
    
    report = pd.DataFrame({
        "daily_active_users": daily_active_users,
        "daily_revenue": daily_revenue
    }).fillna(0)
    
    return report

daily_report = daily_kpi_report(sessions, purchases)
daily_report.head(20)

C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\911013050.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  sessions["session_date"] = pd.to_datetime(sessions["session_date"]).dt.date
C:\Users\Admin\AppData\Local\Temp\ipykernel_4204\911013050.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  purchases["purchase_date"] = pd.to_datetime(purchases["purchase_date"]).dt.date


,daily_active_users,daily_revenue
2025-01-01,2.0,0.0
2025-01-13,12.0,199.0
2025-01-14,17.0,2090.0
2025-01-15,17.0,896.0
2025-01-16,18.0,497.0
2025-01-17,13.0,745.0
2025-01-18,8.0,397.0
2025-01-19,13.0,397.0
2025-01-20,13.0,99.0
2025-01-21,4.0,99.0


In [ ]:
## A/B Testing Analysis
## Statistical Significance (t-test)

data = {
    "user_id": range(1,41),
    "group": ["A"]*20 + ["B"]*20,
    "returned_d1": [1,0,1,1,0,1,0,1,1,0,  1,1,0,1,0,1,0,1,1,0,
                    1,1,1,0,1,1,0,1,1,1,  1,0,1,1,1,0,1,1,1,1]
    }


df = pd.DataFrame(data)

In [ ]:
## ttest_ind stands for independent 2 sample t test
from scipy.stats import ttest_ind

## This creates a boolean mask:
group_a = df[df["group"] == "A"]["returned_d1"]
group_b = df[df["group"] == "B"]["returned_d1"]

t_stat, p_value = ttest_ind(group_a, group_b)
p_value

0.17614019947137752

Interpret p-value

p < 0.05 → Statistically significant difference ✅

p ≥ 0.05 → Difference may be random ❌

**ttest_ind** tests whether the average (mean) of two independent groups is significantly different or not.
👉 ttest_ind answers - Is the difference in retention/ARPU/session time real, or could it be due to random chance?

Ex:
A retention = 24.1%
B retention = 25.0%

But…
👉 Is B actually better?
Or did it just get lucky because of randomness?

ttest_ind helps you avoid fooling yourself with noisy data.

This 0.05 threshold means:

There is less than 5% probability that this difference happened by chance.

In [13]:
## Example in Game Analytics

group_a = [120, 150, 80, 90, 110, 100]
group_b = [140, 160, 130, 120, 155, 145]

t_stat, p_value = ttest_ind(group_a, group_b)

print("t-stat:", t_stat)
print("p-value:", p_value)



t-stat: -2.811607785577666
p-value: 0.018424010056191247


This is less than 0.05, so:

👉 The difference between Group A and Group B is statistically significant.

When should you use ttest_ind?

Use it when:

✅ Two independent groups
✅ Numeric metric (revenue, session time, number of sessions, etc.)
✅ You want to compare means

Key Assumptions (mention this in interviews to sound sharp)

**ttest_ind assumes:**

1️⃣ The two groups are independent
2️⃣ The metric is roughly normally distributed (or sample is large enough)
3️⃣ Variance is similar in both groups (or you can use Welch’s t-test)

❌ When NOT to blindly use ttest_ind

For binary metrics like:

Retained (0/1)

Converted (0/1)